In [1]:
cd ..

/Users/vijay/vijaygkd/char-rnn


In [2]:
%load_ext autoreload
%autoreload 2

# Imports

In [3]:
import numpy as np

import torch
import torch.nn as nn
from torchinfo import summary

/Users/vijay/Library/Caches/pypoetry/virtualenvs/char-rnn-sx3s2mN4-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [107]:
from data.data_utils import process_corpus, str_to_ids, char_idx_to_str
from train.train import train

from model.lstm import LSTM_CharLM
from model.model_utils import sample_token

# Model Exp

## 1. LSTM

In [13]:
m = LSTM_CharLM(128)

summary(m, input_size=(1,50), dtypes=[torch.long])

Layer (type:depth-idx)                   Output Shape              Param #
LSTM_CharLM                              [1, 50, 128]              --
├─Embedding: 1-1                         [1, 50, 12]               1,536
├─LSTM: 1-2                              [1, 50, 128]              72,704
├─Linear: 1-3                            [1, 50, 128]              16,512
Total params: 90,752
Trainable params: 90,752
Non-trainable params: 0
Total mult-adds (M): 3.65
Input size (MB): 0.00
Forward/backward pass size (MB): 0.11
Params size (MB): 0.36
Estimated Total Size (MB): 0.47

In [122]:
# initial value of loss function for untrained model.
# loss = cross entropy for randomly selecting a token from char set of size 128
-np.log(1/128)

4.852030263919617

### Toy sample
Overfit 1 data point

In [113]:
roger = """Roger Federer is widely considered to be one of the greatest tennis players of all time. He has won a record 20 Grand Slam singles titles, including 8 Wimbledon, 6 Australian Open, 5 US Open, and 1 French Open titles. He has also won the ATP Tour Finals a record 6 times, and has been ranked world No. 1 for a record 310 weeks, including 237 consecutive weeks from 2004 to 2008. Federer's all-around game, characterized by his exceptional shot-making, movement, and court coverage, has been called one of the greatest examples of "complete" tennis. He is also known for his longevity, with many of his records coming in his 30s and after. In addition to his success on the court, Federer is also known for his sportsmanship, and is widely respected by both fans and fellow players. Off the court, he is a philanthropist and has been involved in various charitable organizations throughout his career."""

In [123]:
#data
# corpus = [
#     "This text is written by Vijay Gaikwad :)",
#     "Ruchira is my beautiful wife.",
#     "We both happily live in Seattle."
# ]

corpus = [roger[:200]]


X, Y = process_corpus(corpus, seq_len=len(corpus[0])+1)

print(X.shape)

torch.Size([1, 201])


In [124]:
m = LSTM_CharLM(8)

In [125]:
train(m, X, Y, X_test=X, epochs=3000, lr=0.01)

  0%|▎                                                                                              | 9/3001 [00:00<01:12, 41.49it/s]

-----------------
Epoch: 0 | Loss: 4.967576503753662
} WxWWWxxWxW WW.xWWWWWWWWWWWWWWWWWW}xWWW}WW}}xWWWWWWWWWW.WWWWWWWWWW W}WW}WWWWxWWW}}}WWWWW WW .WWWWWx}Wx}WWWWWW}WWWWWWWWW}WWWWWW


 17%|███████████████▊                                                                             | 509/3001 [00:10<00:53, 46.57it/s]

-----------------
Epoch: 500 | Loss: 1.2563029527664185
Roger tldere  ts pinere ton  sele  tinoe oferon tie ore nld loid in tlayers tf tle tine. te oam ton tncedone t0 crand O ameiisg ed tinled  tscous ng o tisbleden  t tusilelenn Open, 5 5S tpen, tnd O tl$


 34%|██████████████████████████████▉                                                             | 1009/3001 [00:22<00:46, 43.06it/s]

...

 67%|█████████████████████████████████████████████████████████████▍                              | 2006/3001 [00:46<00:26, 37.14it/s]

-----------------
Epoch: 2000 | Loss: 0.20960144698619843
Roger Federer is widely considere  ti be one of the greatest tennis players of all time. He has won a record 20 Grand Slam singles titles, including 8 Wimbledon, 6 Austrelean Open, 5 US Open, and 1 Fr$


 83%|████████████████████████████████████████████████████████████████████████████▊               | 2504/3001 [00:59<00:15, 32.45it/s]

-----------------
Epoch: 2500 | Loss: 0.3797396421432495
Roger Federer is pidely cons dere  ti be ofe of tre oreatls eeennis wlayers if all time. He han win a iecors 20 Grand Slam tingles titles, including 8 Wimbledon, 6 tustreledn Open, 5 US Open, and S Ar$


-----------------
Epoch: 3000 | Loss: 0.10870976001024246
Roger Federer is widely considered to be one of the greatest tennis players of all time. He has won a record 20 Grand Slam singles titles, including 8 Wimbledon, 6 Australean Open, 5 US Open, and 1 Fr$


In [126]:
sample_token(m)

'$Roger Federer is widely considere gles 20 Grand Slam singles titles, including 8 Wimbledo 8 Wimbledo'

### Toy Coffee Dataset

Toy coffee corpus containing 64+8 englist sentences about coffee generated by godfather ChatGPT. 
Model is trained to 64 sentences, remaining 8 are used to test the model's language ability.

In [27]:
# load data
coffee_data_path = 'data_files/coffee.txt'

with open(coffee_data_path) as f:
    coffee_corpus = f.read().splitlines()

print(f'Corpus size: {len(coffee_corpus)}')

# max seq len
lens = []
for l in coffee_corpus:
    lens.append(len(l))
print(f'max seq len: {max(lens)}')


Corpus size: 72
max seq len: 98


In [23]:
coffee_corpus[:5]

['The aroma of freshly brewed coffee filled the air, making my mouth water.',
 'She stirred a dollop of cream into her coffee, watching as it swirled and dissolved.',
 'The dark, rich flavor of the coffee was a perfect pick-me-up on a sluggish morning.',
 'He sipped his coffee slowly, savoring the bold, earthy taste.',
 'The barista expertly crafted a latte, with a delicate design of foam on top.']

In [29]:
# train / test split
seq_len = 100

X_train, Y_train = process_corpus(coffee_corpus[:64], seq_len=seq_len)
X_test, Y_test = process_corpus(coffee_corpus[64:], seq_len=seq_len)

print('Size of training data')
print(X_train.shape, Y_train.shape)

Size of training data
torch.Size([64, 100]) torch.Size([64, 100])


In [31]:
# Test samples to expect
coffee_corpus[64:]

['The aroma of freshly brewed coffee was a perfect way to get the day started.',
 "He couldn't resist the temptation of the freshly cursed coffee and had to have a cup.",
 'The barista carefully poured the coffee into a mug and added a touch of chocolate shavings on top.',
 "I couldn't wait to take a sip of my piping hot coffee while enjoying the morning breeze.",
 'The coffee shop was the perfect place to take a break and enjoy a cup of coffee.',
 'The coffee had a smooth and rich flavor, perfect for a morning pick-me-up.',
 'They sat outside, sipping their coffee and enjoying the peaceful atmosphere.',
 'The coffee beans were sustainably sourced and had a unique taste.']

In [37]:
# Train model.
coffee_model = LSTM_CharLM(128)

train(coffee_model, X_train, Y_train, X_test, epochs=3000, lr=0.001)

  0%|                                                                                               | 3/3001 [00:00<04:40, 10.70it/s]

-----------------
Epoch: 0 | Loss: 4.864503383636475
XyXynn}11 n1;1;10bX\1n4}0}0}1}11;10111;nnnnU}0}}nnn1nnnnnb}nnn0n}11nnn }}0}nnnnnnnnnnnnnnnnnnnnnnnn
XX01\1n}}B4}n}0b1;nnnn0nn011}nn 1;nb;1nyyn;}0bX\1n\n}bbb1}11;1011;b1X 1nnbny nynnn\n\nnnnnnnnnnnnnnn
XyXyn4n} ;n n} }0;}}bn\1n}b}1}y0n\11;1011;}bn n nnnn;bnn1}bb11nnnn}Xn1;1\X1X1b n0nnX n ; ;nn;nnbnnn
Xbn\1n}}B4}nnn nnnnnnn;0nnnn nnn;111nn1n1Bnnn1nn}1;;b011Xt\0n0B411 B nnn0n11bB B n4}0000ynnnnnnnnnnn
XyXyn\1;;1011X;1nnnnnnnynn0}0}}n1\ \01}1nnn;0nnn4}0n;nn;b10B4b1n n\nnnn;1\1;;10ynnnnnnnnnnnnnnnnnnn
XyXyn\1;;101X1111nn 1bnynn;bn}11nX\ ;;b1nn0}0}}n1b11n11bB B nn1\0B}0B}}nnnnnnnnnnnnnnnnnnnnnnnnnn
XyXy1nn nnnnnn bb1nn 1\1Bnnnn01bn\11;1011;b10B4b1 B nnn0nn0n10X;}nnn 1;nXy00nnnnnnnnnnnnnnnnnnnnnnnn
XyXyn\1;;101401;;nnb}01nn;n  ; 4}bnnbn}}0}11;b1X 1nnnn;  n}n}nnn0nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn


 17%|███████████████▌                                                                             | 502/3001 [00:55<05:24,  7.71it/s]

-----------------
Epoch: 500 | Loss: 1.201188564300537
The cndft tf creshey boewed coffee aasta serfect aas ao tndhche cac aiatee  $                       
Te could 't wedttt whe terlee enf tf che freshly bopetd coffee and aa  ao te t ancop $              
The corent  sor   lly borted ahe aoffee and  t toc t d an    cnco teypf coaf ffne aie eng .on th  $ 
T could 't west wh te   t sip of co poptng tof aoffee aaell tnjoy ng the tor eng toeaaee$           
The coffee bhef aastaoe terfec  foash bh hh   t sees  t d arjou a sop af coffee $                   
The coffee bad a silf h a d aoso prever  aorfec  frfecncof ing tocoiuedut $                         
The  aas wfth n d aip ing the n coffee and anjoy ng the corferuletneer ee   $                       
The coffee baand aade fapt  tg ey teltee  and al  a spdn t ah te $                                  


 33%|██████████████████████████████▋                                                             | 1002/3001 [01:59<03:55,  8.47it/s]

...

 67%|█████████████████████████████████████████████████████████████▎                              | 2002/3001 [04:04<02:05,  7.99it/s]

-----------------
Epoch: 2000 | Loss: 0.21407106518745422
The croma of freshly brewed coffee was a perfect way to s macoe uir aiurie. $                       
Te couldn't resist the temptation of the freshly bursed coffee tnd tad to have a cup.$              
The carista serefully poured the coffee anto a muc and edded a touch nf foamuuatenaaalorg..pusso .$ 
T couldn't wait to take a sip of my piping hot coffee while wnjoy dg tee ror,.lg.tre..el$           
The coffee bhop was the perfect place to gese t seed  mnd nnjoyia cup of coffee.$                   
The coffee bad a smooth and weth roavor  pirfect for ansorning fick-me-up.$                         
The  sat outside, sipping their coffee and enjoying the baamanuelioeyd eed  $                       
The coffee beans were flgt rs   o tuurler cbo had t snwq e taste $                                  


 83%|████████████████████████████████████████████████████████████████████████████▋               | 2502/3001 [05:04<01:01,  8.15it/s]

-----------------
Epoch: 2500 | Loss: 0.132221981883049
The croma of freshly brewed coffee was a perfect way to s macos uir auurii. $                       
Te couldn't hesist the temptation of the freshly bursed coffee tnd tad to have a cup.$              
The carista sarefully poured the coffee into a muc and edded a touch of foamhuatenaaaloog .ou!oop.$ 
T couldn't wait to take a sip of my piping hot coffee while wnjoy dg theiior..lg..r.dded$           
The coffee bhop was the perfect place to geke t seed  mnd nnjoyia cup of coffee.$                   
The coffee bad a smooth and beth aoavor  pirfect for a morning pick-me-up.$                         
The  sat outside, sipping their coffee and enjoying the baamanueliooyd eed d$                       
The coffee beans were flgc rs   o duurlenfcndnhad b snique taste $                                  


-----------------
Epoch: 3000 | Loss: 0.09461163729429245
The croma of freshly brewed coffee was a perfect way to s macos uir turrii. $                       
Te couldn't resist the temptation of the freshly bursed coffee,tnd tad to have a cup.$              
The carista sarefully poured the coffee into a mug and edded a touch of foamhuatenaaaloog .ou!oop.$ 
T couldn't wait to take a sip of my piping hot coffee while wnjoyidg theiior .ng.pyid  d$           
The coffee bhop was the perfect place to geke t seed  mnd nnjoyia cup of coffee.$                   
The coffee bad a smooth and weth alavor  pirfect for a morning pick-me-up.$                         
The  sat outside, sipping their coffee and enjoying the baamanueliooyd eed  $                       
The coffee beans were fhgc rt  oe duurleregno had a cnwq e taste $                                  


In [110]:
context = "My coffee"
sample_token(coffee_model, context)

'$My coffee shed and to eat efect ansing their coffee wasmeam on top.$   w capmy mumor a uth a picif cup of cof'